In [ ]:
# libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Epydemix import
from epydemix.population import Population
from epydemix.epimodel import EpiModel, stochastic_simulation
from epydemix.plotting import plot_quantiles

# Population Object

In [ ]:
population = Population(name="Indonesia")

# add contact matrices
population.add_contact_matrix(np.load("./basins/Indonesia/contacts-matrix/contacts_matrix_work.npz")["arr_0"], 
                              layer_name="work")
population.add_contact_matrix(np.load("./basins/Indonesia/contacts-matrix/contacts_matrix_home.npz")["arr_0"], 
                              layer_name="home")
population.add_contact_matrix(np.load("./basins/Indonesia/contacts-matrix/contacts_matrix_community.npz")["arr_0"], 
                              layer_name="community")
population.add_contact_matrix(np.load("./basins/Indonesia/contacts-matrix/contacts_matrix_school.npz")["arr_0"], 
                              layer_name="school")

# add population in different age groups
population.add_population(Nk=pd.read_csv("./basins/Indonesia/demographic/Nk.csv")["value"].values, 
                          Nk_names=pd.read_csv("./basins/Indonesia/demographic/Nk.csv")["group"].values)

# EpiModel Object

In [ ]:
# create model 
model = EpiModel(compartments=["S", "R"])

# add compartments
model.add_compartments("I")

# add parameters
model.add_parameters({"beta": 0.04, "mu": 0.1})

# add transitions 
model.add_transition(source="S", target="I", rate_name="beta", agent="I")
model.add_transition(source="I", target="R", rate_name="mu")

# add interventions
model.add_intervention(layer_name="work", start_date="2020-01-01", end_date="2020-05-01", reduction_factor=0.3)
#model.add_intervention(layer_name="school", start_date="2020-01-01", end_date="2020-02-01", new_matrix=population.contact_matrices["community"])


In [ ]:
compartments, df_quantiles = model.simulate(population=population, 
                              start_date="2019-12-01", 
                              end_date="2020-06-01", 
                              S=population.Nk - np.ones(len(population.Nk)),
                              I=np.ones(len(population.Nk)),
                              R=np.zeros(len(population.Nk)),
                              steps=100)

In [ ]:
plot_quantiles(df_quantiles, compartments=["I", "S", "R"], demographic_groups="total")


# Calibration

In [2]:
from scipy import stats 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Epydemix import
from epydemix.population import Population
from epydemix.epimodel import EpiModel, stochastic_simulation
from epydemix.plotting import plot_quantiles
from epydemix.calibration import calibration

# create model 
model = EpiModel(compartments=["S", "I", "R"])
model.add_parameters({"beta": 0.04, "mu": 0.1})
model.add_transition(source="S", target="I", rate_name="beta", agent="I")
model.add_transition(source="I", target="R", rate_name="mu")

# population 
population = Population(name="Indonesia")
population.add_contact_matrix(np.load("./basins/Indonesia/contacts-matrix/contacts_matrix_work.npz")["arr_0"], layer_name="work")
population.add_contact_matrix(np.load("./basins/Indonesia/contacts-matrix/contacts_matrix_home.npz")["arr_0"], layer_name="home")
population.add_contact_matrix(np.load("./basins/Indonesia/contacts-matrix/contacts_matrix_community.npz")["arr_0"], layer_name="community")
population.add_contact_matrix(np.load("./basins/Indonesia/contacts-matrix/contacts_matrix_school.npz")["arr_0"], layer_name="school")
population.add_population(Nk=pd.read_csv("./basins/Indonesia/demographic/Nk.csv")["value"].values, 
                          Nk_names=pd.read_csv("./basins/Indonesia/demographic/Nk.csv")["group"].values)


# initial conditions
S=population.Nk - np.ones(len(population.Nk))
I=np.ones(len(population.Nk))
R=np.zeros(len(population.Nk))

# simulation dates
steps = 100
start_date="2019-12-01"
end_date="2020-06-01"
start_date, end_date = pd.to_datetime(start_date), pd.to_datetime(end_date)
simulation_dates = pd.date_range(start=start_date, end=end_date, periods=steps).tolist()
model.compute_contact_reductions(population, simulation_dates)

# parameters
parameters = {"Cs": model.Cs, 
              "Nk": population.Nk, 
              "S": S, 
              "I": I, 
              "R": R, 
              "epimodel": model}
parameters.update(model.parameters)
results = stochastic_simulation(parameters)

In [ ]:
simulations, sampled_params = calibration(simulation_function=stochastic_simulation, 
                                          parameters=parameters,
                                          priors={"beta": stats.uniform(0.01, 0.03), 
                                                  "mu": stats.uniform(0.1, 0.25)}, 
                                                )

In [6]:
errors = [1,2,4,5,5,12,4,4,5]
np.argwhere(np.array(errors) < 4).ravel()

0